Leung Wai Liu <br>
JPMC-SMM4H <br>
July 14, 2022 <br>
Task 1a Test Majority Ensembling - Roberta Only

In [11]:
import pandas as pd
import numpy as np
from collections import Counter
from labels_to_ids import task7_labels_to_ids
from sklearn.metrics import accuracy_score, classification_report, f1_score, precision_score, recall_score, confusion_matrix
from training_code import calculate_overall_performance_metrics
import os

In [12]:
# Loading up all the predictions data

n_rnds = 3
original_df = pd.read_csv('../Datasets/Subtask_1a/test.tsv', sep='\t')
models = ['roberta-large']
n_models = len(models)

epoch_string = '../2nd_try_15_epochs_large_model/saved_test_result_1a'
n_rows = len(original_df)

labels_to_ids = task7_labels_to_ids
ids_to_labels = dict((v,k) for k,v in labels_to_ids.items())



In [13]:
# Retrieving all the predictions from the 
list_of_df = []

for model in models: 
    specific_model_row = []
    for rnd in range(n_rnds):
        to_read_string = epoch_string + '/' + model + '/' + str(rnd) + '/unformatted_test_result.tsv'
            
        particular_model_df = pd.read_csv(to_read_string, sep='\t')
        specific_model_row.append(particular_model_df)
    
    list_of_df.append(specific_model_row)

In [14]:
# TAKING THE MAJORITY OF DATA

majority_original_tweet_id_list = []
majority_original_sentence_list = []
majority_original_numbered_label_list = []

majority_predicted_number_results = []
majority_predicted_results = []


for index, row in original_df.iterrows(): 
    # getting the original values in the tweet
    original_tweet_id = row['tweet_id']
    original_sentence = row['tweet']
    
    # transferring the labels over to final list
    majority_original_tweet_id_list.append(original_tweet_id)
    majority_original_sentence_list.append(original_sentence)
    
    specific_row_results = []
    # go through every models' row of data 
    
    for model_num in range(n_models):
        for rnd_num in range(n_rnds):
            # print(list_of_df[model_num][rnd_num])
            row = list_of_df[model_num][rnd_num].loc[(list_of_df[model_num][rnd_num]['tweet_id'] == original_tweet_id)]

            prediction = labels_to_ids[row['class'].values[0]]
            specific_row_results.append(prediction)

    specific_row_results = Counter(specific_row_results)
    specific_row_results = specific_row_results.most_common(1)[0][0]
    majority_predicted_results.append(ids_to_labels[specific_row_results])
    

In [15]:
# Saving it as a dataframe
unformatted_majority_prediction_data = pd.DataFrame(zip(majority_original_tweet_id_list, majority_original_sentence_list, majority_predicted_results), columns=['id', 'text', 'class'])
print(unformatted_majority_prediction_data)

formatted_majority_prediction_data = unformatted_majority_prediction_data.drop(columns = ['text'])
print(formatted_majority_prediction_data)

                              id  \
0      SMM4H2022e4OIETplIkjV0I8V   
1      SMM4H2022ZMATVYdmmKWGkRoh   
2      SMM4H2022V61Tilw8DJvqNqLT   
3      SMM4H2022mKSqbziPej1yvp4X   
4      SMM4H2022RV9KzDljn78QqAqi   
...                          ...   
10964  SMM4H2022ecknKTI3uRThqNXt   
10965  SMM4H2022pLCtFgOQqJaU8i3O   
10966  SMM4H2022kSh6JvmRUcyR1lzz   
10967  SMM4H2022i1YwnECBKPiDqTvR   
10968  SMM4H2022NVFJ38j8nXAx5uab   

                                                    text  class  
0      oh, that spiriva is too much $...how bout not ...  noADE  
1      @USER___________ @USER_____ I found NSAIDS did...  noADE  
2      Having delicious Tysabri 4 lunch. Then off 2 @...  noADE  
3      the glorification of adderall during finals is...  noADE  
4      holy shit the 10mg saphris tablet tastes infin...  noADE  
...                                                  ...    ...  
10964  you know it's finals week when you find addera...  noADE  
10965  Zostavax has to be reconstituted

In [16]:
# Saving it as a tsv file
os.makedirs('../2nd_try_15_epochs_large_model/final_test_result', exist_ok=True)
unformatted_majority_prediction_data.to_csv('../2nd_try_15_epochs_large_model/final_test_result/unformatted_majority_data.tsv', sep='\t', index=False)
formatted_majority_prediction_data.to_csv('../2nd_try_15_epochs_large_model/final_test_result/formatted_majority_data.tsv', sep='\t', index=False, header=False)